<a href="https://colab.research.google.com/github/mirrormouse/machine_learning/blob/main/Literature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語の文学を機械学習して新たな文章を生成できないか？
今回は名作を機械学習に用いて新しく「それっぽい文」を生成することを目指します。  
markovify等のマルコフ連鎖を使ったやり方が有名な課題ではありますが、ここはあくまで「機械学習」でやってみます。  
「とりあえずやってみて機械学習に慣れること」が目標なので、あまり難しいことは考えずにいきましょう。

ドライブのインポート

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


形態素解析に必要なものをインストール

In [5]:
!pip install mecab-python3
!pip install unidic

     |████████████████████████████████| 488 kB 5.2 MB/s 
  Created wheel for unidic: filename=unidic-1.0.3-py3-none-any.whl size=5506 sha256=0b456a8c39cf56bb8cbec52525a126be8367a73484b2680b9172cd9cf647532a
  Stored in directory: /root/.cache/pip/wheels/23/30/0b/128289fb595ef4117d2976ffdbef5069ef83be813e88caa0a6
Successfully built unidic


In [6]:
!python -m unidic download

download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic.zip
Dictionary version: 2.3.0+2020-10-08
unidic.zip: 100% 608M/608M [00:30<00:00, 19.7MB/s]
Finished download.
Downloaded UniDic v2.3.0+2020-10-08 to /usr/local/lib/python3.7/dist-packages/unidic/dicdir


必要なものをインポート

In [172]:
import MeCab
import unidic
import random
from keras.utils.np_utils import to_categorical  
import numpy as np
tagger = MeCab.Tagger() 

元となるデータをインストール  
（今回は青空文庫より作・アントワーヌ・ド・サン＝テグジュペリ　訳・大久保ゆうの「星の王子様」の全文を使用しました。名作ですね）

In [88]:
base='drive/MyDrive/ML/train/'
f = open(base+'literature.txt', 'r')
text_data = f.read()
f.close()

Mecabで単語に分解し、リストwordsに入れていきます。["","ぼく","が","６","つ","の","とき"] みたいな感じ。

In [89]:
def mecab_tokenizer(text,window):
    parsed_text = tagger.parse(text)
    parsed_lines = parsed_text.split("\n")
    words=[]
    words.append("")
    for line in parsed_lines:
      word=line.split("\t")
      if word[0]=='EOS':
        break
      words.append(word[0])
      if word[0]=="。":
        words.append("")
    return words
#mecab_tokenizer(text_data,5)

setを使って重複を取り除き、idと単語を対応付ける辞書型リストを作ります。また、テキストを単語に分解したものをidのリストへと変換します。

In [101]:
def preprocess(text_data,window=5):
  text=mecab_tokenizer(text_data,window)
  words=list(set(text))
  word_dic={}
  for id in range(len(words)):
    word=words[id]
    word_dic[word]=id
  vec=[]
  for word in text:
    vec.append(word_dic[word])
  return text,vec,words,word_dic
text,vec,words,word_dic=preprocess(text_data)
print(vec[:10])

[0, 815, 967, 1039, 1802, 153, 408, 1755, 2018, 1725]


one_hotエンコーディングする関数をつくります。  
また、「直前の５単語」をヒントに「次の単語」を推測するモデルを作るための訓練データを作成します。  
例）  
x_train[0]=["","ぼく","が","６","つ"]  y_train[0]=["の"]  
x_train[1]=["ぼく","が","６","つ","の"]  y_train[1]=["とき"]

In [127]:
def one_hot_encode(data,size=-1):
  vec=np.array(data)
  if size==-1:
    size=vec.max()+1
  one_hot = np.zeros((vec.size, size))
  one_hot[np.arange(vec.size),vec] = 1
  np_list=one_hot.astype(int)
  res=np_list.tolist()
  return res
def generate_data(text_data,window=5):
  text,vec,id2word,word2id=preprocess(text_data,window)
  data=one_hot_encode(vec)
  x_list=[]
  y_list=[]
  for id in range(window,len(data)):
    y_list.append(vec[id])
    x=[]
    for i in range(id-window,id):
      x.append(data[i])
    x_list.append(x)
  x_data=np.array(x_list)
  y_data=np.array(y_list)
  return x_data,y_data,id2word,word2id
x_train,y_train,id2word,word2id=generate_data(text_data)

データのshapeを確認します

In [116]:
print(x_train.shape)
print(x_train[0])
print(y_train.shape)

(27495, 5, 2019)
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(27495,)


最初のレイヤーのinput_shape及び最後のレイヤーの引数を、上で確認したshapeに合わせることに注意して、ニューラルネットワークのモデルを作成します。
本当はもっと深くしたかったのですが、単語が2019種類と多くこのモデルでも学習に10分近くかかってしまったので諦めます。
多分出現頻度の低い単語をひとまとめにするとかして単語の種類数を減らすべきだったんでしょうね。しかし今回は「とりあえずやってみる」が目標なので細かいことは無視します。

In [98]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
model = tf.keras.Sequential(name='my_model')
model.add(tf.keras.layers.Flatten(input_shape=(5,2019), name='flatten_layer_1'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2019, activation='softmax'))

model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_layer_1 (Flatten)    (None, 10095)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1292288   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1290      
_________________________________________________________________
dropout_7 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 2019)              22209     
Total params: 1,315,787
Trainable params: 1,315,787
Non-trainable params: 0
________________________________________________

モデルをコンパイルします。とりあえずadamを選びました。適当です。sgdとかでもいいかも知れません。他の引数もこれ以外知らないのでこれを選んだ、という感じです。

In [99]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

学習します！　epochsは適当に少し大きめにしましたが、結果見る限りもう少し小さくても良かったかも。validation_splitはデータの何割を訓練ではなく評価に用いるか、です。適当に0.1とか0.2を選びます。

In [100]:
history = model.fit(x_train, y_train, batch_size=1000, epochs=100,
                    validation_split=0.2)

Epoch 1/100
22/22 [==============================] - 6s 245ms/step - loss: 7.5809 - accuracy: 0.0431 - val_loss: 7.5211 - val_accuracy: 0.0827
Epoch 2/100
22/22 [==============================] - 5s 219ms/step - loss: 7.3127 - accuracy: 0.0352 - val_loss: 6.9241 - val_accuracy: 0.0065
Epoch 3/100
22/22 [==============================] - 5s 221ms/step - loss: 6.2191 - accuracy: 0.0186 - val_loss: 5.4702 - val_accuracy: 0.0469
Epoch 4/100
22/22 [==============================] - 5s 218ms/step - loss: 5.2788 - accuracy: 0.0529 - val_loss: 5.1464 - val_accuracy: 0.0698
Epoch 5/100
22/22 [==============================] - 5s 221ms/step - loss: 5.0916 - accuracy: 0.0757 - val_loss: 5.0751 - val_accuracy: 0.0891
Epoch 6/100
22/22 [==============================] - 5s 218ms/step - loss: 5.0087 - accuracy: 0.0762 - val_loss: 5.0353 - val_accuracy: 0.0987
Epoch 7/100
22/22 [==============================] - 5s 220ms/step - loss: 4.9437 - accuracy: 0.0728 - val_loss: 5.0024 - val_accuracy: 0.0849

学習は結構時間がかかるので、せっかくだし保存しておきます。

In [193]:
model.save("my_model")

INFO:tensorflow:Assets written to: my_model/assets


保存したものを使いたくなった時はロードします

In [195]:
model = tf.keras.models.load_model("my_model")

実際にテキストを入れてpredictを行いたいところですが、テキストではなくonehotベクトルのデータを入れないとpredictしてくれないので、テキストをonehotベクトルに変換する関数を作ります。

In [132]:
def text2onehot(text_data,size=-1):
  parsed_text = tagger.parse(text_data)
  parsed_lines = parsed_text.split("\n")
  words=[]
  words.append(0)
  for line in parsed_lines:
    word=line.split("\t")
    if word[0]=='EOS':
      break
    try:
      words.append(word2id[word[0]])
    except:
      words.append(0)
    if word[0]=="。":
      words.append(0)
  data=one_hot_encode(words,size)
  return data
start=text2onehot("ぼくが６つ",2019)
x_test=[]
x_test.append(start)
print(np.array(x_test).shape)
predictions = model.predict(x_test)
print(predictions)

(1, 5, 2019)
[[1.1311749e-04 3.0126655e-06 7.5805507e-04 ... 1.1883900e-05
  3.9733976e-08 5.0571103e-02]]


無事predictできました。

これでラストです！  
まず4から11行目の所で、与えられたテキストをpredictに入れられる形に直します。  
具体的にはonehotベクトルが５つ並んだ形の２次元リストvecです。  
その後はx_test=[vec]としてpredictを行い、結果から次の単語のidであるnext_idを決定、
そのidに対応するonehotベクトルを生成してvecの末尾に加え、vecの先頭を削除し、
for文の初めに戻る、というのを繰り返します。これで初めの５単語から次々に
次の単語を推測して文を作っていくことができます。  
next_idの決定方法ですが、argmaxで普通に最大値の引数を取ってくる方法をとると、ある５つの単語に対して常に同じ単語を返すことになってしまい、一度「ループ」にはまってしまうとずっと同じ分を繰り返してい仕舞います。（そうでなくても同じ入力に対し常に同じ出力というのは面白くない）  
そこで、randomによって乱数を生成し、次の単語が確率的に決まるようにしています。  
例)  
predictの結果が[0.1, 0.05, 0.2, 0.6, 0.05]のとき、内側のfor文のvalの値は、  
0.1->0.15->0.35->0.95->1.00と増えていきます。
このとき、「valの値が初めて乱数rand以上になったときのid」をnext_idとして採用することにします。例えばrandが0.3なら、next_id=2となります。こうすれば、確率0.6でnext_id=3となり、確率0.05でnext_id=4となる、というようにpredictの結果をそのまま確率として利用することができます。

In [192]:
def make_sentence(start_text,length=10,modelsize=2019,window=5):
  res=start_text
  vec=text2onehot(start_text,modelsize)
  if len(vec)<window:
    for i in range(window-len(vec)):
      one_hot = np.zeros(modelsize, dtype = int)
      one_hot[0]=1
      one_hot_list=one_hot.tolist()
      vec.insert(0,one_hot_list)
  else:
    vec=vec[:5]
  for i in range(length):
    x_test=[]
    x_test.append(vec)
    predictions = model.predict(x_test)
    #next_id=np.argmax(predictions[0])
    rand=random.random()
    val=0
    next_id=modelsize-1
    for i in range(modelsize):
      val+=predictions[0][i]
      if(rand<=val):
        next_id=i
        break
    res+=id2word[next_id]
    one_hot = np.zeros(modelsize, dtype = int)
    one_hot[next_id]=1
    one_hot_list=one_hot.tolist()
    vec.append(one_hot_list)
    vec.pop(0)
  return res
make_sentence("あしたはきっと",50)

'あしたはきっとかんだった。「さばくをふしぎはていねいに王子くん。王子くんは２にんも、ほんとは90１らの、しんじなくもんけど、王子くんはいいよ。」と王子くんがとっても'

結果としては微妙ですね・・・。まあ、モデルも浅いですしword2vecでやるべきいくつかの前処理も難しそうなものはすべて飛ばしてしまったので仕方ありません。精度の改善は今後の課題として、今回はこのあたりにしておきましょう。

###参考文献
「Python の NumPy 配列でのワンホットエンコーディング」https://www.delftstack.com/ja/howto/numpy/one-hot-encoding-numpy/
「あのときの王子くん」https://www.aozora.gr.jp/cards/001265/files/46817_24670.html